In [1]:
cd ..

/home/aisg/ssd/Clean_Code_WS


In [2]:
#source: https://www.kaggle.com/bhaveshsk/getting-started-with-titanic-dataset/data
#data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

#data visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#machine learning packages
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [3]:
train_df = pd.read_csv("./input/train.csv")
test_df = pd.read_csv("./input/test.csv")
df = pd.concat([train_df,test_df], sort=True)

df.head()

FileNotFoundError: [Errno 2] File b'./input/train.csv' does not exist: b'./input/train.csv'

In [ ]:
from src.preprocessing import add_derived_title

df = add_derived_title(df)

df['Title'] = df['Title'].map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}).fillna(0)

In [ ]:
freq_port = df.Embarked.dropna().mode()[0]
df['Embarked'] = df['Embarked'].fillna(freq_port)

In [ ]:
# EXERCISE 2: Write a unit test and extract the following implementation into a function: 
# df = impute_nans(df, columns)

# 'Fare' column
df['Fare'] = df['Fare'].fillna(df['Fare'].dropna().median())

# 'Age' column
df['Age'] = df['Age'].fillna(df['Age'].dropna().median())

In [26]:
%load_ext autoreload
%autoreload 2
from src.workshop import impute_nan

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'src'

In [21]:
df = impute_nan(df, df.columns)

NameError: name 'impute_nan' is not defined

In [ ]:
df['Sex'] = df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

In [ ]:
df['AgeBand'] = pd.cut(df['Age'], 5)

df.loc[ df['Age'] <= 16, 'Age'] = 0
df.loc[(df['Age'] > 16) & (df['Age'] <= 32), 'Age'] = 1
df.loc[(df['Age'] > 32) & (df['Age'] <= 48), 'Age'] = 2
df.loc[(df['Age'] > 48) & (df['Age'] <= 64), 'Age'] = 3

df = df.drop(['AgeBand'], axis=1)

In [ ]:
# EXERCISE 3: Write a unit test and extract the following implementation into a function: add_is_alone_column(df)
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

df['IsAlone'] = 0
df.loc[df['FamilySize'] == 1, 'IsAlone'] = 1

In [ ]:
# drop unused columns
df = df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
df = df.drop(['Ticket', 'Cabin'], axis=1)
df = df.drop(['Name', 'PassengerId'], axis=1)

In [ ]:
df['Age*Class'] = df.Age * df.Pclass

In [ ]:
df['Embarked'] = df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [ ]:
df['FareBand'] = pd.qcut(df['Fare'], 4)

df.loc[ df['Fare'] <= 7.91, 'Fare'] = 0
df.loc[(df['Fare'] > 7.91) & (df['Fare'] <= 14.454), 'Fare'] = 1
df.loc[(df['Fare'] > 14.454) & (df['Fare'] <= 31), 'Fare']   = 2
df.loc[ df['Fare'] > 31, 'Fare'] = 3
df['Fare'] = df['Fare'].astype(int)

df = df.drop(['FareBand'], axis=1)

In [ ]:
train_df = df[-df['Survived'].isna()]
test_df = df[df['Survived'].isna()]
test_df = test_df.drop('Survived', axis=1)

In [ ]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.copy()

In [ ]:
# EXERCISE 1: Create a function, train_model(...), to eliminate the duplication in the next few cells. 

In [ ]:
def train_model(model_type, X_train, Y_train):
    model = model_type()
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    acc = rounding(model.score(X_train, Y_train), 2)
    return model, acc

def rounding(score, deci_pl):
    rounded_score = round(score * 100, deci_pl)
    return rounded_score

svc, acc_svc = train_model(SVC, X_train, Y_train)
knn, acc_knn = train_model(KNeighborsClassifier, X_train, Y_train)
gaussian, acc_gaussian = train_model(GaussianNB, X_train, Y_train)
perceptron, acc_perceptron = train_model(Perceptron, X_train, Y_train)
random_forest, acc_random_forest = train_model(RandomForestClassifier, X_train, Y_train)
sgd, acc_sgd = train_model(SGDClassifier, X_train, Y_train)
decision_tree, acc_decision_tree = train_model(DecisionTreeClassifier, X_train, Y_train)

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)